In [3]:
import pandas as pd
import numpy as np
import re
import tqdm
from tqdm import tqdm 

# Train-test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

# Analisis NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer # stemmer en español
import spacy # lematizador en español
from sklearn.linear_model import LogisticRegression
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances 
import string
from collections import Counter
from textblob import TextBlob
from googletrans import Translator

In [2]:
df = pd.read_csv('data/df_coments_extract.csv')
df

,Rating,Comentario,Url,Timestamp
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038
...,...,...,...,...
391971,Valoración: 3 estrellas,"El barrio era genial, fantásticas zonas para c...",https://www.airbnb.es/rooms/14352069/reviews?a...,2024-11-10 13:46:09.279394
391972,Valoración: 4 estrellas,Muy bien,https://www.airbnb.es/rooms/14352069/reviews?a...,2024-11-10 13:46:09.288397
391973,Valoración: 5 estrellas,Hermoso apartamento nuevo con acceso cercano a...,https://www.airbnb.es/rooms/14352069/reviews?a...,2024-11-10 13:46:09.363919
391974,Valoración: 4 estrellas,Alex fue muy receptivo y las fotos de su aloja...,https://www.airbnb.es/rooms/14352069/reviews?a...,2024-11-10 13:46:09.390927


## **Limpieza**

### **Poner todas las columnas en minúscula**

In [7]:
df.columns = df.columns.str.lower()

### **Eliminamos columnas con comentarios vacíos**

In [9]:
df = df.dropna(subset=['comentario'])

In [10]:
df.shape

(391919, 4)

### **Nos quedamos con el rating (valor numérico)**

In [13]:
df['rating'] = df['rating'].str.extract(r'(\d+)').astype(int)

C:\Users\jesus\AppData\Local\Temp\ipykernel_43268\2610869063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].str.extract(r'(\d+)').astype(int)


In [14]:
df.head(5)

,rating,comentario,url,timestamp
0,5,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516
1,5,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027
2,5,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032
3,5,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035
4,5,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038


## **comentarios**

### *limpiar saltos de línea , puntuacion y emojis*

In [25]:
def clean_coments_df(df,text_column):

    def clean_coments(text):
    
        # Eliminamos saltos de línea
        text = text.replace('\n', ' ').replace('\r', ' ').strip()
    
        # Eliminar emojis
        text = re.sub(r'[^\x00-\x7F]+', '', text).strip()
    
        # Eliminar signos de puntuación
        text = text.translate(str.maketrans('', '', string.punctuation)).strip()
    
        # Convertir todo a minusculas
        text = text.lower()
    
        # Eliminar espacios múltiples
        text = re.sub(r'\s+', ' ', text).strip()
    
        return text

    df[text_column] = df[text_column].astype('str').apply(clean_coments)


    return df

In [27]:
df = clean_coments_df(df,'comentario')

C:\Users\jesus\AppData\Local\Temp\ipykernel_43268\2715314131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df[text_column].astype('str').apply(clean_coments)


In [29]:
# Comprobamos si hay caracteres despues del apply
def detectar_no_permitidos(texto, permitidos="abcdefghijklmnopqrstuvwxyz0123456789 "):
    
    no_permitidos = [c for c in texto.lower() if c not in permitidos]
    return no_permitidos



In [31]:
permitidos = "abcdefghijklmnopqrstuvwxyz0123456789 "
df['caracteres_no_permitidos'] = df['comentario'].apply(lambda x: detectar_no_permitidos(x, permitidos))

# Filtrar filas problemáticas
problemas = df[df['caracteres_no_permitidos'].apply(len) > 0]
print(problemas)


Empty DataFrame
Columns: [rating, comentario, url, timestamp, caracteres_no_permitidos]
Index: []


C:\Users\jesus\AppData\Local\Temp\ipykernel_43268\999950372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['caracteres_no_permitidos'] = df['comentario'].apply(lambda x: detectar_no_permitidos(x, permitidos))


In [32]:
df_group = df.groupby(by = 'url').agg( rating_promedio = ('rating', 'mean'),
                                      cantidad_comentarios = ('comentario' ,'count'),
                                      comentarios = ('comentario' ,  lambda x: list(x))).reset_index()

In [35]:
df_group

,url,rating_promedio,cantidad_comentarios,comentarios
0,https://www.airbnb.es/rooms/100044781045691589...,4.645161,31,[apartamento bien situado para visitar barcelo...
1,https://www.airbnb.es/rooms/100051458827470719...,4.318182,44,[el departamento es bueno para el precio y la ...
2,https://www.airbnb.es/rooms/100114773709033483...,4.842105,57,[todo fue muy agradable en el departamento de ...
3,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u..."
4,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u..."
...,...,...,...,...
2451,https://www.airbnb.es/rooms/998945864657673961...,4.478261,69,[haba mucho ruido del exterior no haba papel h...
2452,https://www.airbnb.es/rooms/999180200821552889...,4.693878,49,[tuve una estancia estupenda en casa de viola ...
2453,https://www.airbnb.es/rooms/999188656384515552...,4.659574,47,[el propietario fue muy amable la casa era rel...
2454,https://www.airbnb.es/rooms/999287710180353527...,4.700000,20,[realmente me encant mi estancia en el lugar d...


In [90]:
%%time

# Lematizador
lematizer_esp = spacy.load('es_core_news_sm')


# Set de stopwords
stop_words = set(stopwords.words('spanish'))

# Traductor
translator = Translator()

# Procesar comentarios de forma individual
def procesar_comentarios_individuales(comentarios, url_idx= None, total_urls= None):
    
    resultados = []
    word_counter = Counter()

        # Mostrar progreso de URLs si se proporciona el índice y total
    if url_idx is not None and total_urls is not None:
        print(f"Procesando URL {url_idx + 1} de {total_urls}")


    for comentario in tqdm(comentarios, desc="Accediendo a los comentarios"):  #comentarios:
        
        # Tokenización
        tokens = word_tokenize(comentario.lower())
        
        # Remover stopwords y caracteres no alfabéticos
        tokens_filtrados = [token for token in tokens if token not in stop_words]
        
        # Lematización
        doc = lematizer_esp(" ".join(tokens_filtrados))
        lemas = [token.lemma_ for token in doc]

        # Actualizar el contador de palabras para este grupo
        word_counter.update(lemas)
        
       # Análisis de sentimiento (traducimos al inglés para usar TextBlob)
        try:
            #sentimiento = TextBlob(comentario).translate(to='en').sentiment
            #polaridad = sentimiento.polarity
            #subjetividad = sentimiento.subjectivity

            traduccion = translator.translate(comentario, src='es', dest='en').text
            blob = TextBlob(traduccion)
            sentimiento = blob.sentiment
            polaridad = blob.polarity
            subjetividad = blob.subjectivity

        except Exception as e:
            # Si falla la traducción, asignamos valores nulos 
            polaridad = 0
            subjetividad = 0

        
        # Guardar resultados de cada comentario
        resultados.append({
            "comentario_original": comentario,
            "tokens": len(tokens),
            "tokens_filtrados": len(tokens_filtrados),
            "lematizacion": len(lemas),
            "polaridad": polaridad,
            "subjetividad": subjetividad,
        })

    most_used_words = word_counter.most_common(10)  # 10 palabras más usadas

   
    return {
        "resultados_individuales": resultados,
        "palabras_mas_usadas": most_used_words
    }


CPU times: total: 406 ms
Wall time: 547 ms


In [92]:
# Obtener el total de URLs
total_urls = len(df_group)

# Aplicar la función con progreso por URL
df_group['resultados'] = df_group.apply(
    lambda row: procesar_comentarios_individuales(
        comentarios=row['comentarios'], 
        url_idx=row.name,  # Índice de la fila actual
        total_urls=total_urls
    ), 
    axis=1
)

Procesando URL 1 de 2456


Accediendo a los comentarios: 100%|██████████| 31/31 [00:04<00:00,  6.81it/s]


Procesando URL 2 de 2456


Accediendo a los comentarios: 100%|██████████| 44/44 [00:06<00:00,  6.46it/s]


Procesando URL 3 de 2456


...

Accediendo a los comentarios: 100%|██████████| 47/47 [00:42<00:00,  1.12it/s]


Procesando URL 2455 de 2456


Accediendo a los comentarios: 100%|██████████| 20/20 [00:19<00:00,  1.00it/s]


Procesando URL 2456 de 2456


Accediendo a los comentarios: 100%|██████████| 41/41 [00:37<00:00,  1.08it/s]


## Polaridad y Subjetividad promedio

In [264]:
df_group['polaridad_media'] = df_group['resultados'].apply( lambda x : sum(comentario['polaridad'] for comentario in x['resultados_individuales']) / len(x['resultados_individuales']) if x['resultados_individuales'] else 0)

df_group['subjetividad_media'] = df_group['resultados'].apply( lambda x : sum(comentario['subjetividad'] for comentario in x['resultados_individuales']) / len(x['resultados_individuales']) if x['resultados_individuales'] else 0)

In [274]:
df_group[['polaridad_media', 'subjetividad_media']]

,polaridad_media,subjetividad_media
0,0.353185,0.575261
1,0.290322,0.552236
2,0.442936,0.621841
3,0.409962,0.596344
4,0.409962,0.596344
...,...,...
2451,0.281087,0.504597
2452,0.368379,0.540729
2453,0.484402,0.678031
2454,0.396096,0.609168


## Palabras mas usadas

In [285]:
df_test = df_group['resultados'][3]

In [289]:
palabras_mas_usadas = df_test['palabras_mas_usadas']

In [307]:
df_group['palabras_mas_usadas'] = df_group['resultados'].apply( lambda x : x['palabras_mas_usadas'])

In [319]:
df_group.head()

,url,rating_promedio,cantidad_comentarios,comentarios,resultados,polaridad_media,subjetividad_media,palabras_mas_usadas
0,https://www.airbnb.es/rooms/100044781045691589...,4.645161,31,[apartamento bien situado para visitar barcelo...,{'resultados_individuales': [{'comentario_orig...,0.353185,0.575261,"[(apartamento, 19), (est, 16), (limpio, 15), (..."
1,https://www.airbnb.es/rooms/100051458827470719...,4.318182,44,[el departamento es bueno para el precio y la ...,{'resultados_individuales': [{'comentario_orig...,0.290322,0.552236,"[(bien, 17), (ubicacin, 16), (apartamento, 14)..."
2,https://www.airbnb.es/rooms/100114773709033483...,4.842105,57,[todo fue muy agradable en el departamento de ...,{'resultados_individuales': [{'comentario_orig...,0.442936,0.621841,"[(apartamento, 24), (estancia, 21), (limpio, 2..."
3,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u...",{'resultados_individuales': [{'comentario_orig...,0.409962,0.596344,"[(limpio, 12), (est, 9), (lugar, 8), (cerca, 5..."
4,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u...",{'resultados_individuales': [{'comentario_orig...,0.409962,0.596344,"[(limpio, 12), (est, 9), (lugar, 8), (cerca, 5..."


## Calificacion de sentimiento

In [328]:
def clasificar_sentimiento(polaridad, subjetividad):
    # Clasificación basada en polaridad
    if polaridad > 0.1:
        sentimiento = "Positivo"
    elif polaridad < -0.1:
        sentimiento = "Negativo"
    else:
        sentimiento = "Neutral"
    
    return sentimiento

# Crear una nueva columna 'sentimiento' aplicando la función
df_group['sentimiento'] = df_group.apply(
    lambda row: clasificar_sentimiento(row['polaridad_media'], row['subjetividad_media']),
    axis=1
)


In [338]:
df_group

,url,rating_promedio,cantidad_comentarios,comentarios,resultados,polaridad_media,subjetividad_media,palabras_mas_usadas,sentimiento
0,https://www.airbnb.es/rooms/100044781045691589...,4.645161,31,[apartamento bien situado para visitar barcelo...,{'resultados_individuales': [{'comentario_orig...,0.353185,0.575261,"[(apartamento, 19), (est, 16), (limpio, 15), (...",Positivo
1,https://www.airbnb.es/rooms/100051458827470719...,4.318182,44,[el departamento es bueno para el precio y la ...,{'resultados_individuales': [{'comentario_orig...,0.290322,0.552236,"[(bien, 17), (ubicacin, 16), (apartamento, 14)...",Positivo
2,https://www.airbnb.es/rooms/100114773709033483...,4.842105,57,[todo fue muy agradable en el departamento de ...,{'resultados_individuales': [{'comentario_orig...,0.442936,0.621841,"[(apartamento, 24), (estancia, 21), (limpio, 2...",Positivo
3,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u...",{'resultados_individuales': [{'comentario_orig...,0.409962,0.596344,"[(limpio, 12), (est, 9), (lugar, 8), (cerca, 5...",Positivo
4,https://www.airbnb.es/rooms/100133620739616372...,4.800000,15,"[limpio y cerca del metro, gracias cesar por u...",{'resultados_individuales': [{'comentario_orig...,0.409962,0.596344,"[(limpio, 12), (est, 9), (lugar, 8), (cerca, 5...",Positivo
...,...,...,...,...,...,...,...,...,...
2451,https://www.airbnb.es/rooms/998945864657673961...,4.478261,69,[haba mucho ruido del exterior no haba papel h...,{'resultados_individuales': [{'comentario_orig...,0.281087,0.504597,"[(apartamento, 34), (bien, 22), (limpio, 21), ...",Positivo
2452,https://www.airbnb.es/rooms/999180200821552889...,4.693878,49,[tuve una estancia estupenda en casa de viola ...,{'resultados_individuales': [{'comentario_orig...,0.368379,0.540729,"[(est, 21), (amable, 21), (casa, 18), (viola, ...",Positivo
2453,https://www.airbnb.es/rooms/999188656384515552...,4.659574,47,[el propietario fue muy amable la casa era rel...,{'resultados_individuales': [{'comentario_orig...,0.484402,0.678031,"[(ubicacin, 26), (lugar, 18), (excelente, 16),...",Positivo
2454,https://www.airbnb.es/rooms/999287710180353527...,4.700000,20,[realmente me encant mi estancia en el lugar d...,{'resultados_individuales': [{'comentario_orig...,0.396096,0.609168,"[(lugar, 10), (habitacin, 10), (apartamento, 1...",Positivo


In [341]:
df_group.to_csv('data/df_nlp.csv', index=False)

In [5]:
df_check = pd.read_csv('data/df_nlp.csv')

In [40]:
df_check.iloc[140][4]

C:\Users\jesus\AppData\Local\Temp\ipykernel_24960\3826307158.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_check.iloc[140][4]


"{'resultados_individuales': [{'comentario_original': 'te recomiendo este alojamiento todo es ms o menos como se muestra en las fotos el personal es amable y est dispuesto a ayudar en cualquier duda o problema lo nico que me sorprendi un poco fue la necesidad de pagar un impuesto en el momento de la llegada la cantidad no es muy alta pero no se mencion en ningn lugar por lo que me cogi desprevenido por lo dems todo bien', 'tokens': 73, 'tokens_filtrados': 30, 'lematizacion': 30, 'polaridad': 0, 'subjetividad': 0}], 'palabras_mas_usadas': [('recomeir', 1), ('alojamiento', 1), ('ms', 1), ('menos', 1), ('mostrar', 1), ('foto', 1), ('personal', 1), ('amable', 1), ('est', 1), ('dispuesto', 1)]}"